### Домашнее задание - 3 урок

 Перепишу сначала код, решение домашки помещаю вниз.

 Часть кода переделал -  у меня нет FeautureSelector, вместо него все отрабатывает ColumnSelector. Показалось громоздким иметь оба

In [17]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier 

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold 
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score, roc_auc_score, log_loss, precision_recall_curve

pd.set_option("display.max_rows", 6)

In [20]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/train_case2.csv', "rb") as f1:
    data2 = pd.read_csv(f1, ';')

df = data2.copy()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
# перепишу тип, т.к. пайпалйн preprocessinga ошибку выдавал
df = df.astype({'gender': np.object, 'cholesterol': np.object})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  object 
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  object 
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(10), object(2)
memory usage: 6.9+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [22]:
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [23]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [24]:
# перепишем алгоритм (я переделал его с одним классом на подачу столбца)

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

transfomer_num =  Pipeline([
            ('selector', ColumnSelector(key=continuos_cols)),
            ('standard', StandardScaler())
        ])

    

cat_transformer = Pipeline([
            ('selector', ColumnSelector(key=cat_cols)),
            ('ohe', OHEEncoder(key=cat_cols))
        ])
  

base_transformer = Pipeline([
            ('selector', ColumnSelector(key=base_cols))
        ])


feats = FeatureUnion([('transfomer_num', transfomer_num),
                      ('cat_transformer', cat_transformer),
                      ('base_transformer', base_transformer)
                      ])

#feature_processing = Pipeline([('feats', feats)])


classifier = Pipeline([
    ('preprocessing', feats),
    ('classifier', LogisticRegression(random_state = 1))
])





#----------------------
#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))



classifier.fit(X_train, y_train)


new_pred_train = classifier.predict(X_train)
new_pred_test = classifier.predict(X_test)


print(f'f1_score Train: {f1_score(y_train, new_pred_train)},\nTest: {f1_score(y_test, new_pred_test)}')

CV score is 0.7851260969029223+-0.0062061822843780985
f1_score Train: 0.7077152231235282,
Test: 0.7092400998929718


__Решение ДЗ:__


1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)


Решение № 1 - хитрое (за счет испольщования GridSearch)

In [25]:
# За основу оставим наш объект classifier, обучим три разных модели на параметрах по умолчанию

%%time

params_grid = [{'classifier': [DecisionTreeClassifier(random_state=1)]},
               
               {'classifier': [RandomForestClassifier(random_state=1)]},
               
              {'classifier': [LogisticRegression(random_state=1)]}
               ]

               
grid = GridSearchCV(classifier, params_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_estimator_

CPU times: user 768 ms, sys: 423 ms, total: 1.19 s
Wall time: 22.7 s


Pipeline(steps=[('preprocessing',
                 FeatureUnion(transformer_list=[('transfomer_num',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key=['age',
                                                                                      'height',
                                                                                      'weight',
                                                                                      'ap_hi',
                                                                                      'ap_lo'])),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('cat_transformer',
                                                 Pipeline(steps=[('selector',
                               

Видим что гридсеч выбрал лучшей моделью логистическую регрессию.



__Решение №2 - раздельное сравнение:__

In [26]:
results = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'fscore_libr', 'Precision', 'Recall', 'ROC AUC'])



In [27]:
%%time

classifier_rf = Pipeline([
    ('preprocessing', feats),
    ('classifier', RandomForestClassifier(random_state=1))
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_rf, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std_rf = np.std(cv_scores)
print(f'CV score by RandomForestClassifier is {cv_score}+-{cv_score_std_rf}')


classifier_rf.fit(X_train, y_train)


pred_train = classifier_rf.predict(X_train)
pred_test = classifier_rf.predict(X_test)


preds_proba = classifier_rf.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
fscore_libr = f1_score(y_test, pred_test)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds_proba)

ix = np.argmax(fscore)

results = results.append({
    'model': 'randomforestclass',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'fscore_libr': fscore_libr,
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': auc
}, ignore_index=True)

CV score by RandomForestClassifier is 0.7725336821753542+-0.001330901902132691
CPU times: user 29 s, sys: 343 ms, total: 29.4 s
Wall time: 35 s


In [28]:
%%time

classifier_ctc = Pipeline([
    ('preprocessing', feats),
    ('classifier', CatBoostClassifier(random_state=1))
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_ctc, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std_ctc = np.std(cv_scores)
print(f'CV score by CatBoostClassifier is {cv_score}+-{cv_score_std_ctc}')


classifier_ctc.fit(X_train, y_train)


pred_train_ctc = classifier_ctc.predict(X_train)
pred_test_ctc = classifier_ctc.predict(X_test)

preds_proba = classifier_ctc.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
fscore_libr = f1_score(y_test, pred_test)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds_proba)

ix = np.argmax(fscore)

results = results.append({
    'model': "catboost",
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'fscore_libr': fscore_libr,
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': auc
}, ignore_index=True)


Learning rate set to 0.047017
0:	learn: 0.6801789	total: 21ms	remaining: 20.9s
1:	learn: 0.6677912	total: 33ms	remaining: 16.5s
2:	learn: 0.6567388	total: 42ms	remaining: 13.9s
3:	learn: 0.6466787	total: 50.4ms	remaining: 12.5s
4:	learn: 0.6375922	total: 59ms	remaining: 11.7s
5:	learn: 0.6294114	total: 68.8ms	remaining: 11.4s
6:	learn: 0.6219800	total: 79ms	remaining: 11.2s
7:	learn: 0.6153746	total: 94.7ms	remaining: 11.7s
8:	learn: 0.6091993	total: 104ms	remaining: 11.5s
9:	learn: 0.6036756	total: 114ms	remaining: 11.2s
10:	learn: 0.5990810	total: 123ms	remaining: 11.1s
11:	learn: 0.5944708	total: 133ms	remaining: 11s
12:	learn: 0.5901181	total: 143ms	remaining: 10.8s
13:	learn: 0.5866606	total: 152ms	remaining: 10.7s
14:	learn: 0.5831524	total: 161ms	remaining: 10.6s
15:	learn: 0.5800272	total: 178ms	remaining: 10.9s
16:	learn: 0.5771417	total: 201ms	remaining: 11.6s
17:	learn: 0.5744949	total: 217ms	remaining: 11.9s
18:	learn: 0.5719894	total: 244ms	remaining: 12.6s
19:	learn: 0.56

SVC не хватило терпения ждать, выключил. 

In [29]:
# %%time

# classifier_svc = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', SVC(random_state=1, probability=True))
# ])

# #запустим кросс-валидацию
# cv_scores = cross_val_score(classifier_svc, X_train, y_train, cv=3, scoring='roc_auc')
# cv_score = np.mean(cv_scores)
# cv_score_std_svc = np.std(cv_scores)
# print(f'CV score by SVC is {cv_score}+-{cv_score_std_svc}')


# classifier_svc.fit(X_train, y_train)

# pred_train = classifier_svc.predict(X_train)
# pred_test = classifier_svc.predict(X_test)


# pred_proba = classifier_svc.predict_proba(X_test)[:1]

# precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
# fscore_libr = f1_score(y_test, pred_test)
# fscore = (2 * precision * recall) / (precision + recall)
# auc = roc_auc_score(y_test, preds_proba)

# ix = np.argmax(fscore)

# results = results.append({
#     'model': 'svc',
#     'thresh': thresholds[ix],
#     'F-Score': fscore[ix],
#     'fscore_libr': fscore_libr,
#     'Precision': precision[ix],
#     'Recall': recall[ix],
#     'ROC AUC': auc
# }, ignore_index=True)


In [30]:
%%time

classifier_lg = Pipeline([
    ('preprocessing', feats),
    ('classifier', LogisticRegression(random_state=1))
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_lg, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std_lg = np.std(cv_scores)
print(f'CV score by LogisticRegression is {cv_score}+-{cv_score_std_lg}')


classifier_lg.fit(X_train, y_train)

pred_train = classifier_lg.predict(X_train)
pred_test = classifier_lg.predict(X_test)


pred_proba = classifier_lg.predict_proba(X_test)[:1]

precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
fscore_libr = f1_score(y_test, pred_test)
fscore = (2 * precision * recall) / (precision + recall)
auc = roc_auc_score(y_test, preds_proba)

ix = np.argmax(fscore)

results = results.append({
    'model': 'LogisticRegression',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'fscore_libr': fscore_libr,
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': auc
}, ignore_index=True)

CV score by LogisticRegression is 0.784389330450559+-0.0003064674901576115
CPU times: user 1.39 s, sys: 962 ms, total: 2.35 s
Wall time: 1.25 s


In [31]:
results

,model,thresh,F-Score,fscore_libr,Precision,Recall,ROC AUC
0,randomforestclass,0.370000,0.729011,0.714951,0.665305,0.806209,0.774156
1,catboost,0.376305,0.750274,0.714951,0.700918,0.807108,0.802472
2,LogisticRegression,0.376305,0.750274,0.709240,0.700918,0.807108,0.802472


4. сделать выводы о том, какая модель справилась с задачей лучше других:<br>

По выведенной выше таблице видим, что CatBoost и LogisticRegression справляются одинаково лучше чем случайный лес


5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? 
(когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?


__Ответ:__ roc_auc_score?